<a href="https://colab.research.google.com/github/StefanOglesby/chatbot/blob/main/V2_SO_experiments_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
1+1
#ERROR IN ROW 457 must be corrected
#IMPORT python-docx REPLACING spire.
# use python-docx: The changes affect line 311 and lines 341 to 411 of the code.
################################################################################
###
###  RAG PROJECT FOR G-I-M: Knowledge management for "qual" discussion guides and
###  "quant" online survey questionnaires.
#### reading, parsing and embedding and querying word and excel documents
################################################################################
#
# The script has 4 SECTIONS:
# Sections I, II and III load and transform and index data for the knowledge base.
# Section IV is the backend for an interactive tool allowing to retrieve knowledge.
#
# SECTION I: reading the .docx and .xlsx files, creating a dictionary w/ metadata
# for each file. For "qual" documents, each paragraph is extracted as a string in
# a list ["text 1"]. For "quant" documents, each row of a table in the word document
# or the excel document is extracted as a list of stings [*cell 1", "cell 2", "cell 3"]
#
#  dict_1 = {"ID": ID, "method": method, "language": language, "country": country,
#        "file_name": file_name, "file_path": file_path, "payload":
#            {"format": "x", "content": [["cell 1", "cell 2", "cell 3"]
#                                           ["cell 1", "cell 2", "cell 3"]]},
#            {"format:" "p", "content": [["text 1"], ["text 2"], ["text 3"]]}}
#
#  formats: p = paragraph in .docx, t = table in .docx, x = .xlsx
#
# SECTION II: extracting the relevant text strings for embedding / indexing.
# For "qual" document, each paragraph is appended to the list of relevant strings.
# For "quant" documents, the questionstrings within the rows are identified
# and only the question-strings are appended to the list of relevant strings.
# Meta-Data for each string selected is added: Document ID, and the location in the
# documents dictionary.
#
# q_list is the list of relevant strings:  ["cell 5", "text 10", "cell 2"]
#
# SECTION III: merging of the strings selected for embedding with metadata that is
# required for filtering documents in the index database during queries, e.g. for method
# ("qual" or "quant"), country or language.
# Embedding the strings using Azure openAI API embedding model, and merging the indexes
# with the document dictionaries and the metadata into a dataframe.
#
# df_merged is te resulting combination of index database and metadata all in one dataframe for
# easy retrieval in the query section.
# For production, the indexes will be stored in an index database,  e.g. qdrant, together
# with payload = metadata that is required for filtering the index database during queries
# , plus an  identifier (ID) for the original document, that allows to retrieve the original
# content related to the results of the query plus additional metadata, such as the link
# to the related .docx or .xlsx document.
#
# SECTIONn IV: submitting queries, getting top 3 matches using cosine similarity, extracting
# and printing top 3 text chunks with metadata (method, file name, file link)


################################################################################
###### I N I T I A L I S A T I O N #############################################
################################################################################
################################################################################

!pip3 install llama_index
!pip3 install llama_index.llms.azure_openai
!pip3 install llama_index.embeddings.azure_openai

!pip3 install sklearn.metrics.pairwise

# We need to install the key for the paid, full version of spire
#############   replaced by pyhton-docx !pip install spire.doc

!pip3 install python-docx
# !pip3 install docx2python

!pip3 install docx2txt

# !pip3 install mistralai

#### for semantic splitting into chunks

# !pip3 install llama_index.node_parser
# !pip3 install llama_index.text_splitter


1+1
# from docx2python import docx2python
## original prototype
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
## load Document in order to manually create llama index document
from llama_index.core import Document
from llama_index.core.llms import ChatMessage
from llama_index.readers.file import DocxReader
from sklearn.metrics.pairwise import cosine_similarity
# # import streamlit as st
# import logging
import pandas as pd
import sys
import os
import numpy as np

import json
## additional for importing documents
import docx2txt

## alternative to spire.doc
import docx
from docx import Document


# from spire.doc import *
# from spire.doc.common import *

# from spire.doc.common import License as docLicense

## Apply license for Spire.Doc
# docLicense.SetLicenseKey(
# "3QEAC+/ahTUBEe5UDEB+a8VG1YfC2FlFhngme5hXo0miCVV4Fpdtcy4UvQ3DC+gK7SLg/5NoiwSI9SW6WVE6pmDrIpEar4HF+ODoinu6pgQPOVHHk0si/3mH/jmLetaQPw15LMkjYRCIFRfMqa2wDiodI7QRPlQVvu6BS6PLgfXB/KEuPtgfVcmREd7R72Mq2JZtVjvxsETNBaCGglur+WHHm/caU24uYHszXcRM2m6kdiJeQaV33I4WxLl7NFTClzaeK62ZFp/+4FcG1F2nc02F1a4abANVKzAMwJQ2EdBHg5/t4iFkNr6Uhb7NyAN5c4J58KTvEkE1Hq/qHA3MnJCKMcfkxU0cYzpZnxdx8QjpZjRveQLCXbXfOuLoAZHgvZtuHOsCeyZQL7WEZCi2BykczBNui4/Kk0GPw6fj3ABY8DHwaupWDLO19WicbekPZjUcyvJymHAuve5nPNf9Oqb6gE1zHVhKHggKPNV4SAKdNm+HtAb8bOuBnmTpaQsemC85wBNnZcFxYzyrLtU82gjcS0yBOcGcxxgL6QRhzowHrPQX1KP0Cye0vt7jCSt6/D4rYVijBZ//s6RklAQFv7/TJzwVlZSLQuLEr6+ve5lDaURdWwRe+tg3aY4VQJOLpfe1wxqBI3Z/9kM+NArVjmcoVDoXf/2wV8rDnWvnKQZjz7Bzjgfkru3PZ8dx19pH/Bsxo+Of0IRTKGJKOpDmmHu/h2oyFgnwJDYz0r65bNv6nDmHLEB/GkKSFWEDTuUNjg3oeb7fWrfGIDGxh0o1Nt/0ZpUYMVURjRo1tYWyY+utHHfySvJQagJOF477wdjWBaFnsq7BwmIVHiLUh4qcxC/8+e3GSUjsr7OaesAwmQLq4nY27zsqfZqsP94hWZbys2NZrztbqblWRVLNmp30FY/1AwA06TumNTw8UBshTT2fzF76KTYzjti70tPFkmcZU9X4q4J8fsbOfJ6FM4MYSoxaswZfms6AKX6IGIK5mf22qKY5QB+gDaQWGExzSdVJ1kq5w2Kaxs6LZaXhTW+FL9F7IjnnfbhRa8tuVoFG5xr4Wa3EpPcFOwMUaBOtM8auFuWFmXgzgRpgynw8CxAOrO2ObnmT8fWuUVGyIuh/JBciXLPaufwlQ+h/wYF2P/jP4swvJ2LzTibK633MfHD3zIMCsZYM3vqPsSol9jSmmAJ/c6ZA5LJW6ONfjg0yCPD5n6I5K/3pQf2dOm8+3arNemwFrVSx2BSbBmbT3fkt37FbHMN7o6VUrX+mh42I7sfIaB7Ee7NUh9O9GupFNgHQi0E2vZTOLNEe75or+GeietsD288Rmb+5lf2ef59SL5hqrU8bqSpKwUpBJEYSwy49L+4Kqd9ZIaSgX0nVN4Lem7+6s8U0hZihmLViKMLJEZ3GYT21TzfghNHq4XTIIp+sLX3qwckC+QraTOxUrkVXcbsGx1b0buADLAPSWUuo1fKUpmwWvVy+YZJUneRF/Ja9lKMcwSnnQIGFGIxfA2HJcmg3JTmvTNqi7lZOij5/jRni/XrAIz4oW/b6DZOO0LLgb6kPsofJVRgD/MnyjM3N1qo2nol+9ZjX3cxU9E+GCa5O5/Tfeuno96JAx3HGzMUSGl+5OnxdIYhjYBpGvjFXDDM=")

# License.SetLicenseKey("license.elic.xml")

####  basics ##############
from google.colab import userdata

from typing import List

## for regular expressions
import re

#   import mistralai

####################################################################################
####################################################################################
## Define functions ################################################################
####################################################################################
###### PROTOTYPE ###################################################################
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')

#######define text splitter into paragraphs (\n\n) and eliminate empty paragraphs
#### NOTE WE NEED TO ADD: STRIP LEADING BLANCS!

def split_text_into_paragraphs(text: str) -> List[str]:
    """Splits a text into paragraphs."""
    return [para.strip("") for para in text.split('\n\n') if para.strip() != '']

##### define function for classifying strings into question or answer using openAI
##### needs defined llm object

def classify_paragraph(prompt: str) -> str:
    """
    Executes a chat completion using the provided prompt and returns the result.
    """
    system_prompt = """You are a survey researcher and your task is to classify strings as
                "question", or "answer", or "other". Only answer with "question", "answer", or "other".
                Learn from the following examples:
                Would you describe yourself as the main driver of any car in your household? // question
                Please write down all tyre brands that come to mind. Please enter only one brand in each box. // question
                How much are you interested in buying this product? // question
                Please specify the type of your motorcycle. // question
                What is your age? // question
                I would definitely buy this product // answer
                between 15 and 24 years // answer
                yes // answer
                no // answer
                At a motorcycle dealer // answer
                don't know // answer
                Country relevant for winter/all-season tyre // other
                Country // other
                System variable // other"""
    # Create the messages list with system and user messages
    messages = [
        ChatMessage(role="system", content=system_prompt),
        ChatMessage(role="user", content=prompt),
    ]

    # Execute the chat completion
    try:
       response = llm.chat(messages=messages).message.content
    except:
       response = "error"

    # Extract and return the result from the response
    return response

###############################################################
#### initialize azure openai ##################################
###############################################################

AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')

aoai_api_key = AZURE_OPENAI_KEY
aoai_endpoint = "https://pulsar.openai.azure.com/"
aoai_api_version = "2023-03-15-preview"

llm = AzureOpenAI(
    model="gpt-4-32k",
    deployment_name="pulsar-typr-azure-openai-gpt4-32k-0613",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

Settings.llm = llm
Settings.embed_model = embed_model

################################################################################
###### S E C T I O N  I ########################################################
################################################################################
################################################################################
###############  INITIALISATION ################################################

# all files read will be converted into standardized python dictionaries and
# appended to so_dicts

# in production, the dictionaries can be stored as jason files in sql database

so_dicts = []

################################################################################
################################################################################
################ QUALITATIVE ###################################################
# Qualitativ - Discussin Guides for method = "qual"
# Read ==> docx files from local folder
# Using standard docx reader because no tables and want full document
# Note: We assume that method "qual" is always a word document

# Read files from local folder
folder_path = "/content/drive/MyDrive/qualdata/"
input_files = [
    os.path.join(folder_path, filename)
    for filename in os.listdir(folder_path)
    if filename.endswith(".docx")
 ]

# Read files with simple directory reader
# Result is a list of document objects (llamindex document objects)
documents = SimpleDirectoryReader(input_files=input_files).load_data()
print("\n".join(input_files))
len(documents)


#  CREATE DICTIONARY FROM DOCUMENTS IN LIST OF DOCUMENTS
# inizialize metadata for dictionary
# NOTE: language and country are not available now
# we need to define how country and language are manually added
# in production version, language and country will be queried from ERP database

### so_dicts = []   # for local testing only

method = "qual"
language = "German"
country = "Germany"

#  creat a separate dictionary from each document and append to list
for doc in documents:
   paragraphs = split_text_into_paragraphs(doc.text)
   data_list =  [[string] for string in paragraphs]
   file_name = doc.metadata["file_name"]
   file_path = doc.metadata["file_path"]
   ID = hash(file_name)
   dict_1 = {"ID": ID, "method": method, "language": language, "country": country,
        "file_name": file_name, "file_path": file_path,
             "payload": [{"format": "p", "content": data_list }] }
   so_dicts.append(dict_1)


# experiments
so_dicts[2]["payload"][0]["content"]

len(so_dicts)
# using list comprehension
print([par[0] for par in so_dicts[0]["payload"][0]["content"]])

print("\n\n".join([par[0] for par in so_dicts[1]["payload"][0]["content"]]))

################################################################################
################################################################################
######################## QUANTIATIVE  ##########################################
# Quantitative - Survey Questionnaires WORD   ##################################
# for method = "quant"
# Read ==> docx files from local folder
#
#  PROBLEM: TABLES ARE NOT PRINTED CORRECTLY FROM LLAMANINDEX DOCUMENT OBJECT
#  SOLUTION: python-docx  OPEN SOURCE PACKAGE (REPLACES SPIRE)
#
####################  Extract paragraphs and tables ############################

# LOOP OVER ALL TABLES, EXTRACT ROWS: 1 ROW = 1 LIST
# LOOP OVER ALL PARAGRAPHS, EXTRACT PARAGRAPHS: 1 PARAGRAPH = 1 LIST W/ 1 ELEMENT

################################################################################
### Get input files from directory for reading docx documents
folder_path = "/content/drive/MyDrive/quantdata"

input_files = [filename for filename in os.listdir(folder_path) if filename.endswith(".docx")]


# Load Word documents to a list of documents - SPIRE DOCUMENT OBJECT
so_documents = []
for file_name in input_files:
   file_path = os.path.join(folder_path, file_name)
   document = Document(file_path)
   # document.LoadFromFile(file_path)
   print(f"""sections {len(document.sections)} in document {file_name}""")
   so_documents.append(document)


# EXPERIMENTING
file_path = os.path.join(folder_path, input_files[0])


document = Document("/content/drive/MyDrive/quantdata/Quant_Fragebogen_Bio.docx")

print(input_files[0])
len(so_documents)
document = so_documents[0]
print(document)

tables = document.tables
len(tables)
[tables[3].rows[2].cells[i].paragraphs[0].text for i in range(len(tables[3].rows[2].cells)) ]


################################################################################
# LOOPING THROUGH ALL DOCUMENTS method = "quant" and end ".docx"
# LOOPTING THROUGH ALL PARAGRAPHS AND TABLES IN ALL SECTIONS  ###
# CREATING DICTIONARY FOR EACH document
## append result to so_dicts = all dictionaries   ##############################

#>>>>>>>>>>>>>> TO DO: LOOPING THROUGH documents
# for testing
## document = so_documents[0]

# loop through documents
file_index = 0   # file_index is counter = index in the loop
for document in so_documents:

  # inizialize metadata for dictionary
  folder_path = folder_path
  file_name = input_files[file_index]
  file_path = os.path.join(folder_path, file_name)
  ID = hash(file_name)
  method = "quant"
  language = "German"
  country = "Germany"

  dict_1 = {"ID": ID, "method": method, "language": language, "country": country,
        "file_name": file_name, "file_path": file_path }

  dict_1["payload"] = []

      # get the paragraphs in the document
      # create list of paragraphs - each paragraphs is a list in itself
      # one Text string per paragraph = list
      # all paragraphs of a section in the data_list
  data_list = []

      # loop through the paragraphs of the document
  for i in range(len(document.paragraphs)):
         if document.paragraphs[i].text != '':
            data_list.append([document.paragraphs[i].text])
  dict_1["payload"].append({"format": "p", "content":  data_list})


      # Get the tables in the document
      # each table is a seaparate data_list
      # table rows are list of strings
  tables = document.tables

      # Loop through the tables
  for i in range(0, len(tables)):

          ## do one table only
          table = table = tables[i]
          # Create a list to store the extracted table data
          data_list = []
          data_row = []

          # Loop through the rows in the table
          for j in range(len(table.rows)):
             row = table.rows[j]
             data_row = []
             # Loop through the cells in each row
             for k in range(len(row.cells)):
                cell = row.cells[k]
                # Loop through the paragraphs in each cell
                cell_text = ""
                for p in range(len(cell.paragraphs)):
                   # Extract data from each paragraph
                   paragraph = cell.paragraphs[p]
                   text = paragraph.text
                   # print(f"""{k} ... {text}...""")
                   # Append the data to the list
                   cell_text = cell_text + text

                # print(cell_text)
                # print(f"""{i} ... {cell_text}...""")
                data_row.append(cell_text)
             print(f"""{i} ... {data_row}...""")
             data_list.append(data_row)
          dict_1["payload"].append({"format": "t", "content":  data_list})

  so_dicts.append(dict_1)
  file_index = file_index + 1


### EXPERIMENTING
len(so_dicts)
so_dicts[6]["ID"]
## dict_1["payload"].append({"format": "t", "content":  data_list})
dict_1
document.Close()

dict_1["file_name"]
so_dicts[4]["payload"][8]

## nice print of payload paragraphs and tables
dict_1["payload"][1]["format"]
print(" \n".join(so_dicts[4]["payload"][8]["content"][0]))
for s in so_dicts[3]["payload"][8]["content"]:
    print("\t¦".join(s))


################################################################################
################################################################################
### Quantitative - Survey Questionnaires EXCEL
### Read ==> xlsx files from local folder
####### WITH A DICTIONARY FOR EACH FILE  AS FINAL OUTPUT   #####################

### Get input files from directory for reading docx documents
folder_path = "/content/drive/MyDrive/quantdata"

input_files = [filename for filename in os.listdir(folder_path)
  if filename.endswith(".xlsx")  or filename.endswith('.xls')]
print("\n".join(input_files))

#######  EXTRACT INTO 1  DICTIONARY PER XLS DOCUMENT
#######  LOOP THROUGH ALL XLS FILES AND CREATE ONE DICTIONARY FOR EACH XLS FILE
#######  EACH ROW OF THE EXCEL IS ONE LIST IN A LIST = CONTENT
# file_name = input_files[2]
#min([df.shape[1],15])
# df.shape[1]

## so_dicts = []   # only for local testing of loops
method = "quant"
language = "German"
country = "Germany"

for file_name in input_files:
    file_path = os.path.join(folder_path, file_name)
    ID = hash(file_name)
    # Read  the Excel file into a DataFrame and extract rows as string
    df = pd.read_excel(file_path, engine='openpyxl', dtype=str)
    ### identify first column with question texts up to last column or column 15
    soq = []
    for i in range(10,40):
      for j in range(min([df.shape[1],15])):
        if len(str(df.iloc[i,j])) > 10:
          if "question" == classify_paragraph(df.iloc[i,j]):
              soq.append(j)
              print(j)
              print(df.iloc[i,j])
    if not soq: soq.append[11]
    print(min(soq))
    # replace NaN with string = blanc
    df.fillna('', inplace=True)
    #shorten df columns up to  first column with question text found
    dfs = df.iloc[:,:min(soq)+1]

    # add dataframe to list of dataframes OBSOLETE - USE DICTIONARIES
    # so_dataframes.append(dfs)
    # add dictionary to list of dictionaries
    dicts = {"ID": ID, "method": method, "language": language, "country": country,
                "file_name": file_name, "file_path": file_path,
                 "payload": [{"format": "x", "content":  dfs.values.tolist()}] }
    so_dicts.append(dicts)


# EXPERIMENTING
len(so_dicts)
dict_1 = so_dicts[7]
dict_1["ID"]
dict_1["method"]
dict_1["payload"][0]["format"]
dict_1["payload"][0]["content"]

for s in so_dicts[7]["payload"][0]["content"]:
    print("\t¦".join(s))

dfs.iloc[0:40,:]
df.iloc[0:20,0:13]
# EXPERIMENTING
len(str(df.iloc[10,9]))
classify_paragraph(df.iloc[10,9])
type(df.iloc[10,9])

len(so_dataframes)
df_1 = so_dataframes[2]
df_1.shape[1]

df_1.iloc[0:40,0:11]
so_listx = df_1.values.tolist()
so_listx[28][9]
type(so_listx)
len(so_listx)


classify_paragraph(so_listx[28][9])

type(df_1)

################################################################################
###### S E C T I O N  II #######################################################
################################################################################
################################################################################
## EXTRACTING STRINGS AND ID FROM DICTIONARIES AS INPUT FOR EMBEDDING ##########

## INITIALIZING THE LISTS FOR INPUT DATA FOR EMBEDDINGS INCL. METADATA

d_list = []    # document ID
p_list = []    # payload index
q_list = []    # element = question or paragraph text to be embedded
r_list = []    # element = row of question text in payload p

################################################################################
################################################################################
##  EXTRACTION OF all PARAGRAPHS FROM DICTIONARIES CONTENT FOR METHOD = "qual" #
## for DICTIONARIES: extract all paragraphs  ###################################
## works for word files read with simple paragraph splitter ####################

# experimenting
## if work with one dictionary only extract it from so_dicts
dict_1 = so_dicts[0]
len(so_dicts)
# experimenting
data_list = dict_1["payload"][0]["content"]
dict_1["payload"][0]["format"]
dict_1["payload"][0]["content"]

################################################################################
######  initalize loop  for qual dictionaries only  ############################
##  remove initialization here for full run over qual and quant ################
# only for local testing
##d_list = []    # document ID
#p_list = []    # payload index
#q_list = []    # element = question text
#r_list = []    # element = row of question text in payload p


#### loop through the dictionaries of so_dicts  of method = "qual"
for dict_1 in [i for (i, v) in zip(so_dicts, [dic["method"] == "qual" for dic in so_dicts]) if v]:

##### loop through el = elements of list of lists = rows in payload

  p=0
  for el in dict_1["payload"]:
    print(p)
    print(dict_1["file_name"])
    print(el["format"])
    # flatten q_list into list of strings and extend q_list
    [q_list.extend(sublist) for sublist in el["content"]]
    r_list.extend([x for x in range(0,len(el["content"]))])
    p_list.extend([p] *len(el["content"]) )
    d_list.extend([dict_1["ID"]] * len(el["content"]))
    print(dict_1["payload"][p]["content"])
    p = p+1


## experiment

flat_list=[]
[flat_list.extend(sublist) for sublist in elx["content"]]
len(flat_list)

len(q_list)
len(r_list)
len(p_list)
len(d_list)

mylist = [x for x in range(0,53)]
[1] * 5
elx= dict_1["payload"][0]
mylist2 = [p] * len(elx["content"])
len(mylist2)
mylist3 = [dict_1["ID"]] * len(elx["content"])
len(mylist3)

################################################################################
################################################################################
##  EXTRACTION OF QUESTIONS FROM DICTIONARIES FOR METHOD = "quant" #############
## for DICTIONARIES: extract rows marked as "question" in same row #############
## works for excel files and for tables from word file sections ################

################################################################################
################################################################################
### extract dictionaries with method = "quant" #################################

# so_dicts_quant = [i for (i, v) in zip(so_dicts, [dic["method"] == "quant" for dic in so_dicts]) if v]
len(so_dicts)

# for local testing of loops only
#d_list = []    # document ID
#p_list = []    # payload index
#q_list = []    # element = question text
#r_list = []    # element = row of parapraph text in payload p


##### loop through dictionaries with method = quant

for dict_1 in [i for (i, v) in zip(so_dicts, [dic["method"] == "quant" for dic in so_dicts]) if v]:

##### loop through el = elements of list of lists = rows in payload
  p=0
  print(dict_1["file_name"])
  for el in dict_1["payload"]:
    q_len = 0
    print(p)
    print(el["format"])
    if el["format"] == "t" or el["format"] == "x":
      data_list = el["content"]

      for i in range(len(data_list)):
        q_ind = False   # initialize - indicating a row with "question" or "q"
        row = list(data_list[i])
        if "question" in [x.lower() for x in row]:
          row.pop([x.lower() for x in row].index("question"))
          q_ind = True
        if "q" in [x.lower() for x in row]:
          row.pop([x.lower() for x in row].index("q"))
          q_ind = True
        q_success = False  # initializie - indicating wether row is appended
          ### row[-1] ONLY FOR FORMAT X // DO NOT INSERT SPACE IF row[-1]
        if el["format"] == "x" and q_ind and row[-1].strip() != "":
           q_list.append("".join(row[-1]))
           q_success = True
        elif el["format"] == "t" and q_ind:
           q_list.append(" ".join(row))
           q_success = True
        else:
          pass
          #print(f'{i}The string question was not found in the list.')
        if q_success:
             q_len = q_len + 1
             r_list.append(i-0)
             p_list.append(p)
             d_list.append(dict_1["ID"])
             print(dict_1["payload"][p]["content"][i])

  ###   fallback if no "q" or "question"  found = q_len = 0
  ###   all rows with blanc in row left to last row with questions
  ###   needs improvment  metadata q_found = "yes" if last row is question
      if el["format"] == "x" and q_len < 1:
         for i in range(len(data_list)):
            row = list(data_list[i])
            if bool(re.fullmatch("",row[-2])) and "".join(row[-1]) != "":
              q_list.append("".join(row[-1]))
              r_list.append(i-0)
              p_list.append(p)
              d_list.append(dict_1["ID"])
              print(dict_1["payload"][p]["content"][i])
    else:
      pass

    p=p+1


# EXPERIMENTING
q_len
q_list
len(d_list)
len(p_list)
len(q_list)
len(r_list)

print(q_list)
print(data_list[r_list[21]])
print("\n".join(q_list))
str(data_list[0])


################################################################################
################################################################################
## Experiments only.
# The function classify_paragraph() is created at start of script
# Classify paragraph question, answer, other

# Experiments

prompt = '50-59 Jahre'
prompt = '60-69 Jahre'
prompt = '70 Jahre oder älter'
prompt = 'TOPIC'
prompt = 'Region'
prompt = "Bitte geben Sie nun an, welche der folgenden Supermärkte Sie kennen, wenn auch nur dem Namen nach."
prompt = "Aldi"
prompt = "Alnatura"
prompt = "Wer ist bei Ihnen zu Hause für den Kauf von Lebensmitteln zuständig?"
prompt = "Ich gemeinsam mit anderen Personen (Partner, Eltern, Mitbewohner,…)"
prompt = "Andere Personen (Partner, Eltern, Mitbewohner,...)"
prompt =  "Gestützte Bekanntheit Einkaufsstätten"

classify_paragraph(prompt)


## split text experiments
text_list = ["""How much do you like this product?""","""rather dislike""","""Ich gemeinsam mit anderen Personen (Partner, Eltern, Mitbewohner,…)"""]
so_documents = [Document(text=t) for t in text_list]

para_class = []
## the split_text_into_paragraphs() function is not loaded - c.f. code collection below
for doc in so_documents:
  paragraphs = split_text_into_paragraphs(doc.text)
  for par in paragraphs:
     para_class.append(classify_paragraph(par))

################################################################################
###### S E C T I O N  III  #####################################################
################################################################################
################################################################################
################################################################################
###### ADDING METADATA AND EMBEDDING RELEVANT TEXT STRINGS  ####################
###
### Preparing two auxiliary dataframes for easy access to metadata and content
###
### df_documents: document ID and all dictionaries in one column (= series)
### the dictionaries are contained in the dataframe series = column "Dict"
### in df_documents the metadata is "hidden" in the dictionaries in "Dict"
### in production we will probably store the dictionaries as jason in SQL

df_documents =  pd.DataFrame({"DocumentID":[dict["ID"] for dict in so_dicts],
                              "Dict": so_dicts
                             })
## inspect result
df_documents["DocumentID"]

#############  write json of documents dictionary  #############################
jason_documents = {"DocumentID":[dict["ID"] for dict in so_dicts],
                              "Dict": so_dicts
                             }
## FOR WRITING DOCUMENTS AS JASON CREATE JUST A DICTIONARY
import json

with open("/content/drive/MyDrive/probing_data/jason_documents2.json", "wb") as f:
    f.write(json.dumps(jason_documents).encode("utf-8"))

## df_metadata is redundant, but it allows an easy merge with dataframe df_index
## in production this will probably be taken from an squl database

df_metadata = pd.DataFrame({"DocumentID":[dict["ID"] for dict in so_dicts],
                            "method": [dict["method"] for dict in so_dicts],
                            "country":  [dict["country"] for dict in so_dicts],
                            "language":  [dict["language"] for dict in so_dicts]
                             })

################################################################################
# Generate embeddings and store in vectors list and then add to df_index

# Store  document information
## doc_titles = [] #obsolete
# d_list = []    # document ID
# p_list = []    # payload index
# q_list = []    # element = question or paragraph text
# r_list = []    # element = row of question text in payload p

## initialize df_index with metadata and input strings for embedding
## column with vectors will be added after embedding process is completed

df_index = pd.merge(
                    pd.DataFrame({'DocumentID' : d_list,
                         'PayloadIdx' : p_list,
                         'RowIdx' : r_list ,
                         'Text' : q_list
                         }) ,
                    df_metadata,
                   left_on='DocumentID', right_on="DocumentID", how='left')

len(df_index)


################################################################################
###  FILTER  - THIS WILL BE A DROPDOWN IN THE PROTOYPE APP
###  FILTERS ARE: method (quant/ qual), country, language dbd.
#### for this script:  Filtering lists by metadata. example: method
###  Final solution will use SQL query to data in dictionaries (??)

###  EXAMPLE FOR FILTERING:
## df_index = df_index.loc[df_index["method"] == "quant",:]

################################################################################
### CREATING EMBEDDINGS =  INDEX VECTORS  ######################################
vectors = []
vec_error = embed_model.get_text_embedding("error")

df_index["Text"][0]

for q in df_index["Text"]:
  try:
   vector = embed_model.get_text_embedding(q)
   print(vector[0:4])
   vectors.append(vector)
  except:
   vectors.append(vec_error)
   print("error" + q)

df_index["Vector"] = vectors


#######  WRITE df_index TO JASON FILE ON DRIVE #################################

df_index_as_json_str = df_index.to_json(orient='records')


with open("/content/drive/MyDrive/probing_data/df_index_as_json_str2.json", "wb") as f:
   f.write(json.dumps(df_index_as_json_str).encode("utf-8"))



#EXPERIMENTING
len(vectors)
q_list[q_list == ""]

q_list.index("")
q_list[0]
q_list


################################################################################
###### S E C T I O N  IV  ######################################################
################################################################################
################################################################################
### QUERY  #####################################################################


query = "Alles in allem, wie zufrieden sind Sie insgesamt mit der Suva? "
query = "Do you currently own and run a motorcycle?"
query = "Anforderungen an Wassersprudler"
query = "Warum kaufen Sie die Marke Tesa?"

query_vector = embed_model.get_text_embedding(query)

df = df_index

# Calculate the similarity between the query vector and the document vectors
df["Similarity"] = df["Vector"].apply(
        lambda x: cosine_similarity([x], [query_vector])[0][0]
    )

# Sort the DataFrame by similarity in descending order
df = df.sort_values(by="Similarity", ascending=False)

# Get the top 3 most similar documents
df_top_documents = df.head(3)

## EXPERIMENT
df_top_documents["DocumentID"]
print(df_top_documents["Text"].iloc[0])
print(df_top_documents["Text"].iloc[1])
print(df_top_documents["Text"].iloc[2])

### lookup of dictionary information for the top results - now with dataframe but sql in future
### df_documents contains the original document dictionaries that could be stored as json in SQL
## add document information from df_documents to df_top_documents
df_merged = pd.merge(df_top_documents, df_documents, left_on='DocumentID', right_on='DocumentID', how='left')
len(df_merged)

################################################################################
# PRINTING OUTPUT ##############################################################
## extract original information from merged dataframe
## extract information from document's dictionary

for rslt in range(0,3):
  content = df_merged["Dict"][rslt]["payload"][df_merged["PayloadIdx"].iloc[rslt]]["content"]
  print("Original document:\n",
    "¦\t".join(content[df_merged["RowIdx"].iloc[rslt]]),"\n",
    "¦\t".join(content[df_merged["RowIdx"].iloc[rslt]+1]),"\n",
    "¦\t".join(content[df_merged["RowIdx"].iloc[rslt]+2]),
    "\n\nfile_name:",
    df_merged["Dict"][rslt]["file_name"],
    "\nmethod:",
    df_merged["Dict"][rslt]["method"],
    "\nlink:",
    df_merged["Dict"][rslt]["file_path"],
    "\n\n****************************************************************************\n\n")


[i for i in content]

test_list = sum(content, [])

# EXPERIMENTING
## extract original information from merged dataframe
## extract information from document's dictionary

for rslt in range(0,3):
  print("Original document:\n",
    "¦\t".join(df_merged["Dict"][rslt]["payload"][df_merged["PayloadIdx"].iloc[rslt]]["content"][df_merged["RowIdx"].iloc[rslt]:df_merged["RowIdx"].iloc[rslt]+5]
    ),
    "\n\nfile_name:",
    df_merged["Dict"][rslt]["file_name"],
    "\nmethod:",
    df_merged["Dict"][rslt]["method"],
    "\nlink:",
    df_merged["Dict"][rslt]["file_path"],
    "\n\n****************************************************************************\n\n")


## EXPERIMENT

df_merged["Dict"][0]["payload"][df_merged["PayloadIdx"].iloc[0]]["content"]
[i for i in range(0,3)]
print("Original document:\n",
   "¦\t".join(df_merged["Dict"][0]["payload"][df_merged["PayloadIdx"].iloc[0]]
             ["content"][df_merged["RowIdx"].iloc[0]]
   ),
   "\nfile_name:",
   df_merged["Dict"][0]["file_name"],
   "\nmethod:",
   df_merged["Dict"][0]["method"],
   "\nlink:",
   df_merged["Dict"][0]["file_path"]  )



df_merged["Dict"][0]["payload"][0]["content"][20:25]

type(df_merged["Dict"][0]["payload"][0]["content"])

len(df_merged)

df_top_documents["PayloadIdx"].iloc[0]
df_top_documents["RowIdx"].iloc[0]

type(df_merged["Dict"])

print(df_merged[["Text","Document Link"]].to_string(index=False))

## extract original information from merged dataframe - extract information from document's dictionary
print("Original document:\n\n",
   "¦\t".join(df_merged["Dict"][0]["payload"][df_top_documents["PayloadIdx"].iloc[0]]["content"][df_top_documents["RowIdx"].iloc[0]]
   ),
   "\nfile_name:",
   df_merged["Dict"][0]["file_name"],
   "\nmethod:",
   df_merged["Dict"][0]["method"],
   "\nlink:",
   df_merged["Dict"][0]["file_path"]  )
############ LEARNINGS FOR QDRANT SOLUTION

# The df_paragraphs dataframe is to be stored in the qdrant
# The Document Id may be used as ID in the quadrant collection
# The Metadata for the document can be stored in a SQL database.
# The Metadata for the 3 top results is queried from the SQL database via Document ID
# Important: We need to filter the search in the qdrant collection by relevant metadata:
# examples: type of document (guide, questionnaire), country, language
# The metadata that is used for filters must be stored in the payload of the qdrant collection


###### END ########################################################

first row of the dataframe: content[df_merged["RowIdx"].iloc[rslt]]
second row of the dataframe: content[df_merged["RowIdx"].iloc[rslt]+1]
third row of the dataframe: content[df_merged["RowIdx"].iloc[rslt]+2]

tenth row of the dataframe: content[df_merged["RowIdx"].iloc[rslt]+9]


for i in


 "¦\t".join(content[df_merged["RowIdx"].iloc[rslt]]),"\n",
    "¦\t".join(content[df_merged["RowIdx"].iloc[rslt]+1]),"\n",
    "¦\t".join(content[df_merged["RowIdx"].iloc[rslt]+2]),


13

# **SEMANTIC SPLITTER**


In [ ]:
##########  EXPERIMENTING WITH SEMANTIC SPLITTING

from llama_index.core import ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.text_splitter import SemanticTextSplitter

from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
  )

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=80, embed_model=embed_model
  )

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)


print(nodes[3].get_content())

for idx, node in enumerate(nodes):
    print(f"Chunk {idx+1}:")
    print(node.text)
    print("="*50)  # Separator for readability

# **Function that determines wether a cell contains only an integer**

In [ ]:
### determine paragraphs which contain only an integer = answer code

def contains_only_integer(text: str) -> bool:
    """
    Identifies if the input string contains only an integer (positive or negative) and nothing else.

    :param text: The input string to check.
    :return: True if the string contains exactly an integer and nothing else, False otherwise.
    """
    # Regular expression to match a whole integer (optional negative sign)
    pattern = r'^-?\d+$'

    # Return True if the entire string matches the pattern (contains only an integer)
    return bool(re.fullmatch(pattern, text))


# **Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Chatcompletion with the llamandiex setting.llm = llm**

In [ ]:
prompt = """Who is the president of the United States of America"""

messages = [
    ChatMessage(
        role="system", content="You are friendly assistant"
    ),
    ChatMessage(role="user", content=prompt),
]

response = llm.chat(messages=messages)

response.message.content

print(response)

# **The code used in Typr for using the azure api chat completion**

In [ ]:

###########  this is the code used by typr.ai ####################

chat_completion_args = {
    "model": "gpt-4",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the future of AI in healthcare?"}
    ],
    "temperature": 0.7,
    "max_tokens": 100
   }

response = client_azure_openai.chat.completions.create(**chat_completion_args)

# **Reading table in a word file**

In [ ]:
from spire.doc import *
from spire.doc.common import *

# Create a Document instance
document = Document()
# Load a Word document
document.LoadFromFile("CreateTable.docx")

# Get the first section of the document
section = document.Sections[0]

# Get the first table in the section
table = section.Tables[0]

# Create a list to store the extracted table data
data_list = []

# Loop through the rows in the table
for i in range(table.Rows.Count):
    row = table.Rows[i]
    # Loop through the cells in each row
    for j in range(row.Cells.Count):
        cell = row.Cells[j]
        # Loop through the paragraphs in each cell
        for k in range(cell.Paragraphs.Count):
            # Extract data from each paragraph
            paragraph = cell.Paragraphs[k]
            text = paragraph.Text
            # Append the data to the list
            data_list.append(text + "\t")
    data_list.append("\n")

# Write the extracted data into a text file
with open("TableData.txt", "w", encoding = "utf-8") as text_file:
    for data in data_list:
        text_file.write(data)

document.Close()

# **Full loops extract tables from docx document**

In [ ]:
from spire.doc import *

from spire.doc.common import *

# Create an instance of Document

doc = Document()

# Load a Word document

doc.LoadFromFile("Sample.docx")

# Loop through the sections

for s in range(doc.Sections.Count):

    # Get a section

    section = doc.Sections.get_Item(s)

    # Get the tables in the section

    tables = section.Tables
    # Loop through the tables

    for i in range(0, tables.Count):

        # Get a table

        table = tables.get_Item(i)

        # Initialize a string to store the table data

        tableData = ''

        # Loop through the rows of the table

        for j in range(0, table.Rows.Count):

            # Loop through the cells of the row

            for k in range(0, table.Rows.get_Item(j).Cells.Count):

                # Get a cell

                cell = table.Rows.get_Item(j).Cells.get_Item(k)

                # Get the text in the cell

                cellText = ''

                for para in range(cell.Paragraphs.Count):

                    paragraphText = cell.Paragraphs.get_Item(para).Text

                    cellText += (paragraphText + ' ')

                # Add the text to the string

                tableData += cellText

                if k < table.Rows.get_Item(j).Cells.Count - 1:

                    tableData += '\t'

            # Add a new line

            tableData += '\n'



        # Save the table data to a text file

        with open(f'output/Tables/WordTable_{s+1}_{i+1}.txt', 'w', encoding='utf-8') as f:

            f.write(tableData)

doc.Close()


# **Classification of question, answer, other with llm api**

In [ ]:
##### LAMAINDEX: USING THE CHATCOMPLETION WITH AZURE openAI model #############
###### PURPOSE: CLASSIFICATION OF QUANT SURVEY TEXTS AS Q OR A #################


system_prompt = """You are a survey researcher and your task is to classify strings as
                "question", or "answer", or "other". Only answer with "question", "answer", or "other".
                Learn from the following examples:
                Would you describe yourself as the main driver of any car in your household? // question
                Please write down all tyre brands that come to mind. Please enter only one brand in each box. // question
                How much are you interested in buying this product? // question
                Please specify the type of your motorcycle. // question
                What is your age? // question
                Why did you purchase these tyres? Please select all reasons that apply. // question
                I would definitely buy this product // answer
                between 15 and 24 years // answer
                yes // answer
                no // answer
                At a motorcycle dealer // answer
                don't know // answer"""


prompt = """How much do you like this product?"""
prompt = """yes"""
prompt = """rather dislike"""
prompt = """in the internet"""
prompt = """Where did you look for information about healthy food?"""
prompt = """Ich gemeinsam mit anderen Personen (Partner, Eltern, Mitbewohner,…)"""


messages = [
    ChatMessage(
        role="system", content=system_prompt),
    ChatMessage(role="user", content=prompt),
   ]

response = llm.chat(messages=messages).message.content

result = response.message.content

response.raw.id

response.raw.usage.total_tokens

print(response)

# **Experimenting with excel and type document**

In [ ]:
################################################################################
##########  EXPERIMENTING WITH EXCEL ###########################################

1+1
file_path_1 = "/content/drive/MyDrive/sodata/FB_Test_2020_D_F_I.xlsx"

### file_path_1 = "/content/drive/MyDrive/sodata/Quant_Fragebogen_20230213_CTL.xlsx"

### file_path_1 = "/content/drive/MyDrive/sodata/Quant_Regionaler_KS.xlsx"



# Read the Excel file into a DataFrame
df_1 = pd.read_excel(file_path_1, engine='openpyxl')

df_1.iloc[40:100,0:12]

df_class = df_1.iloc[:,0]
df_text = df_1.iloc[:,9]

## extract columns and create new dataframe with relevant columns only

df_class_text = pd.concat([df_1.iloc[:,0],df_1.iloc[:,1],df_1.iloc[:,2]], axis=1 )

## copy text string from column of answers to column with questions if type = A

df_class_text.iloc[:,1][df_class_text.iloc[:,0] == "A"] = df_class_text.iloc[:,2]

df_test = df_class_text[40:200]

print(df_test)

df_test["class"] = df_test.iloc[:,1].apply(classify_paragraph)

classify_paragraph("wie alt sind sie?")

# for excel files of style linkinternational extract rows with "Q" in first column = question texts
rslt_df = df_1.loc[df_1.iloc[:,0] == "Q"].iloc[:,1]

# for excel files of style GIM mit Question label
rslt_df = df_1.loc[df_1.iloc[:,4] == "Question"].iloc[:,9]

# for excel files of style GIM ohne  Question label
rslt_df = df_1.loc[df_1.iloc[:,8].isnull()].iloc[:,9]

rslt_df = df_1.loc[df_1.iloc[:,8].isnull() & ~df_1.iloc[:,9].isnull()].iloc[:,9]

df_1.iloc[:,8].isnull() & ~df_1.iloc[:,9].isnull()



len(df_1.iloc[:,8].isnull())
len(df_1.iloc[:,9].isnull())

type(df_1.iloc[:,9])

print(rslt_df)
## create strings for document object creation
## paragraph = \n\n before each row for final string

len(rslt_df)

rows_as_string = "\n\n".join(rslt_df.astype(str).values.tolist())
print(rows_as_string)
## rslt_df = df_1.loc[dataframe['Percentage'] != 95]


# construct a document FROM DATAFRAME = EXCEL ROWS
# note: id_ is added automatically to document

so_document_1 = Document(
    text= rows_as_string,
    metadata={"filename": "<doc_file_name>",
              'file_path': file_path_1,
              'file_type': 'xlsx',
              "category": "<category>"},
 )


## construct a document example
so_document_2 = Document(
    text="Das ist mein Text",
    metadata={"filename": "<doc_file_name>",
              'file_path': '/content/drive/MyDrive/sodata/inlinecode.txt',
              'file_type': 'xxxx',
              "category": "<category>"},
 )

text_list = ["dies ist mein Text","und jetzt der zweite Text"]
so_documents = [Document(text=t) for t in text_list]

#########################3
#########################
######### THIS IS THE SOLUTION
so_documents_1 = [so_document_1, so_document_2]

print(so_documents_1[0])
so_documents.metadata['filename']

all_documents = documents + [so_document_1]

len(documents)
len(all_documents)
all_documents[3]
len([so_document_1])
type(so_document_1)
type([so_document_1])

# append does not work as follows:
### do NOT use newlist = list.append("x")

documents.append(so_document_1)
len(documents)

# **Old embedding loops**

In [ ]:
################################################################################
################################################################################
###### EMBEDDING THE INPUT DOCUMENTS OLD APPROACH WITH LLAMINDEX DOCUMENTS #####

### TESTING EXCELS = SO_DOCUMENTS

documents = so_documents

# Store  document information
doc_titles = []
doc_names = []
doc_links = []
doc_1_ids = []

# Store paragraph information
para_texts = []
vectors = []
doc_ids = []


for doc in documents:
    # Get the document information
    file_name = doc.metadata["file_name"]
    doc_title = doc.text.split("\n")[0].strip()
    doc_id = doc.id_
    doc_titles.append(doc_title)
    doc_names.append(file_name)
    doc_link = os.path.join(folder_path, file_name)
    doc_links.append(doc_link)
    doc_1_ids.append(doc_id)
        # Get the vector embedding of the document BY PARAGRAPH
    paragraphs = split_text_into_paragraphs(doc.text)
    for par in paragraphs:
      vector = embed_model.get_text_embedding(par)
      print(vector[0:10])
      vectors.append(vector)
      para_texts.append(par)
      doc_ids.append(doc_id)


len(doc_ids)
print(vectors[1][0])
len(vectors)
len(vectors[0])
type(vectors[0])
len(para_texts)

len(doc_titles)
len(doc_1_ids)


# Create a DataFrame for the documents information (metadata)
df_documents = pd.DataFrame(
    {
        "Document Title": doc_titles,
        "Document Name": doc_names,
        "Document Link": doc_links,
        "Document ID": doc_1_ids,
    }
 )


# Create a DataFrame for the paragraphs embeddings
df_paragraphs = pd.DataFrame(
    {
        "Document ID": doc_ids,
        "Text": para_texts,
        "Vector": vectors,
    }
 )


# **Obsolete Code**

In [ ]:
### creating a llamaindex document
            so_doc = Document(
              text= "\n\n".join(df.astype(str).values.tolist()),
              metadata={"file_name": file_name,
              'file_path': file_path,
              'file_type': 'xlsx',
              "category": "<category>"},
               )
            so_documents.append(so_doc)


1+1

In [ ]:
#######  LOOP THROUGH ALL XLS FILES AND CREATE DOCUMENT FOR EACH XLS FILE
#######  OLD VERSION EXTRACTING QUESTION IN DATAFRAME FOR ALL XLS
## create initial dataframe with 1 row and 1 column
## df = pd.DataFrame({'Text1': [1]})

so_documents = []
folder_path = "/content/drive/MyDrive/sodata/"
for file_name in os.listdir(folder_path):
        # Check if the file is an Excel file (by extension)
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            file_path = os.path.join(folder_path, file_name)

            # Read  the Excel file into a DataFrame and extract rows as string
            df = pd.read_excel(file_path, engine='openpyxl')
            df1 = df.loc[df.iloc[:,0] == "Q"].iloc[:,1]
            df2 = df.loc[df.iloc[:,4] == "Question"].iloc[:,9]
            df3 = df.loc[df.iloc[:,8].isnull() & ~df.iloc[:,9].isnull()].iloc[:,9]
            df_t = pd.concat([df1, df2, df3], ignore_index=True)
            so_doc = Document(
              text= "\n\n".join(df_t.astype(str).values.tolist()),
              metadata={"file_name": file_name,
              'file_path': file_path,
              'file_type': 'xlsx',
              "category": "<category>"},
               )
            so_documents.append(so_doc)

so_documents[0]

In [ ]:
# Read files from local folder
folder_path = "/content/drive/MyDrive/quantdata"
input_files = [
    os.path.join(folder_path, filename)
    for filename in os.listdir(folder_path)
    if filename.endswith(".docx")
 ]



In [ ]:
# Experimented filtering paragraphs = cells with only numbers
only_numbers = []
for s in paragraphs[200:300]:
    only_numbers.append(contains_only_integer(s))

filtered_list = [i for (i, v) in zip(paragraphs[200:300], only_numbers) if not v]
